In [1]:
%cd "/content/drive/MyDrive/Project image 2 latex/crohme/HDF5"

/content/drive/MyDrive/Project image 2 latex/crohme/HDF5


In [2]:
%ls

all_latexes_encodedTEST_CROHME_dataset.json
all_latexes_encodedTRAIN_CROHME_dataset.json
all_latexes_encodedVAL_CROHME_dataset.json
all_latexes_TEST_CROHME_dataset.json
all_latexes_TRAIN_CROHME_dataset.json
all_latexes_VAL_CROHME_dataset.json
all__latex_strings.txt
all_stringd.txt
tokens.tsv
Vocabulary_CROHME_dataset.json


In [15]:
from torch.utils.data import DataLoader,Dataset
from torch import nn
import json
from torchvision import transforms

In [32]:
# lets try resizing first as it can be efficient in batching

class Crohme(Dataset):
  def __init__(self ,transforms,extension = ".inkml.png",split=None ):
    super(Crohme,self).__init__()
    assert split in {"TRAIN" , "VAL" , "TEST"} , "split is not right"

    with open(f"./all_latexes_encoded{split}_CROHME_dataset.json") as f :
      encoded_latexes = json.load(f)

    self.encoded_latexes = encoded_latexes
    self.image_names = [key for key,_ in self.encoded_latexes.items()]
    self.extension = extension
    self.len = len(self.encoded_latexes)
    self.transforms = transforms

     
  def __len__(self):
        return self.len

  def __getitem__(self ,index):
        item = self.image_names[index] + self.extension
        image = Image.open(item)
        # Remove alpha channel
        image = image.convert("RGB")
        if self.transform:
            image = self.transform(image)

        #return  {"name":item , "encoded_latex": encoded_latexes[image_names[index]], "image": image}
        return {"image" :image ,"encoded_latex": self.encoded_latexes[self.image_names[index]] } 






In [17]:
with open("/content/drive/MyDrive/Project image 2 latex/crohme/HDF5/Vocabulary_CROHME_dataset.json") as v :
  vocabulary = json.load(v)

In [18]:
vocabulary["<pad>"]

117

In [33]:
def collate_batch(data):
    max_len = max([len(d["encoded_latex"]) for d in data])
    # Padding with 117 as vocabulary[<pad>] = 117
    padded_encoded = [
        d["encoded_latex"] + (max_len - len(d["encoded_latex"])) * [117]
        for d in data
    ]
    
    return {
        "images": torch.stack([d["image"] for d in data], dim=0),
          "encoded_latexes": torch.tensor(padded_encoded),
        }
    

In [34]:
transformations = transforms.Compose([
                                 transforms.ToTensor() ,
                                 transforms.Resize((100,500)  ,
                                  transforms.Normalize((0.5,0.5,0.5) , (0.5,0.5,0.5)))
])

Crohme_training_dataset = Crohme(transformations , split ="TRAIN")
dataloader = DataLoader(Crohme_training_dataset ,batch_size=32 ,shuffle=True , collate_fn =collate_batch)